In [3]:
import pandas as pd
import numpy as np
import itertools
import re
import datetime as dt
import warnings
warnings.filterwarnings('ignore')
import json

pd.options.display.max_columns=None
pd.set_option('max_colwidth', 120)
pd.set_option('display.width', 500)
#pd.options.display.float_format ='{:,.2f}'.format

In [5]:
day_now, month_now, year_now = dt.date.today().day, dt.date.today().month, dt.date.today().year
adamas = pd.read_csv(fr'C:\Users\Trenkin.Sergey\Desktop\parsers\workspace\adamas_all_{day_now}_{month_now}_{year_now}.csv')

In [7]:
adamas.insert(1, 'shop', 'Adamas')

In [8]:
adamas.head()

,date,shop,group,name,article,weight,size,price_old,price,params,url
0,14.03.2024,Adamas,304,Кольцо,1212150-А501Ф-01,3.30,838.0,97290,58374,"[{'Категория': 304}, {'Тип кольца': 319}, {'Цвет металла': 40}, {'Металл': 8}, {'Вставка': 397}, {'Тематика': 2592},...",https://adamas.ru/catalog/rings/1212150-A501F-01/
1,14.03.2024,Adamas,302,"Цепь, плетение Гарибальди",ЦГРП250СА6-А51,7.91,826.0,60990,60990,"[{'Категория': 302}, {'Вид плетения': 256}, {'Плетение': 145}, {'Цвет металла': 40}, {'Металл': 8}, {'Вставка': 397}...",https://adamas.ru/catalog/chains/TSGRP250SA6-A51/
2,14.03.2024,Adamas,304,Кольцо,12039153/01-А50-01,1.81,838.0,56690,34014,"[{'Категория': 304}, {'Тип кольца': 319}, {'Цвет металла': 40}, {'Металл': 8}, {'Вставка': 397}, {'Пол': 291}, {'Тем...",https://adamas.ru/catalog/rings/12039153-01-A50-01/
3,14.03.2024,Adamas,304,Обручальное кольцо из красного золота 585 пробы с бриллиантом,1254553/01-А50Д-41,2.38,821.0,162490,64996,"[{'Категория': 304}, {'Тип кольца': 319}, {'Цвет металла': 40}, {'Металл': 8}, {'Вставка': 59}, {'Обработка': 296}, ...",https://adamas.ru/catalog/rings/1254553-01-A50D-41/
4,14.03.2024,Adamas,305,Серьги из красного золота 585 пробы с бриллиантом,2457923/01-А507Д-41,3.71,NaN,64490,64490,"[{'Категория': 305}, {'Тип серег': 321}, {'Цвет металла': 40}, {'Металл': 8}, {'Вставка': 59}, {'Обработка': 294}, {...",https://adamas.ru/catalog/earrings/2457923-01-A507D-41/


In [9]:
def dict_params(x):
    dict_list = eval(x)
    result = {}
    for d in dict_list:
        result.update(d)
    return result
adamas['params'] = adamas['params'].apply(dict_params)

In [10]:
params_keys = list(set(itertools.chain(*[list(p.keys()) for p in adamas['params']])))

In [11]:
set(params_keys)

{'',
 'Вес',
 'Вид',
 'Вид замка',
 'Вид плетения',
 'Вставка',
 'Вставки',
 'Знак зодиака',
 'Категория',
 'Количество камней',
 'Коллекция',
 'Металл',
 'Обработка',
 'Плетение',
 'По случаю',
 'Пол',
 'Стиль',
 'Тематика',
 'Тип браслета',
 'Тип кольца',
 'Тип пирсинга',
 'Тип подвески',
 'Тип серег',
 'Цвет металла'}

In [12]:
adamas[['color_metal', 'metal', 'insert']] = np.nan
for i in adamas.index:
    par_keys = adamas['params'][i].keys()
    try:
        if 'Цвет металла' in par_keys:
            adamas['color_metal'][i] = adamas['params'][i]['Цвет металла']
        if 'Металл' in par_keys:
            adamas['metal'][i] = adamas['params'][i]['Металл']
        if 'Вставка' in par_keys:
            adamas['insert'][i] = adamas['params'][i]['Вставка']
        if 'Вставки' in par_keys:
            adamas['insert'][i] = adamas['params'][i]['Вставки']
    except:
        if 'Вставка' in par_keys:
            try:
                adamas['insert'][i] = ', '.join(map(str, adamas['params'][i]['Вставка']))
            except Exception as e:
                print(i)
                print(e)

In [13]:
group_dict = {
    304: 'Кольца',
    302: 'Цепи',
    305: 'Серьги',
    303: 'Браслеты',
    306: 'Подвески',
    307: 'Колье',
    308: 'Броши',
    311: 'Пирсинг',
    315: 'Слиток золота'
}

In [14]:
for i in adamas['group'].unique():
    if i not in group_dict.keys():
        print(i)

In [15]:
adamas.query("group == 315")

,date,shop,group,name,article,weight,size,price_old,price,params,url,color_metal,metal,insert


In [16]:
adamas['group_new'] = np.nan
for i in adamas.index:
    adamas['group_new'][i] = group_dict[adamas['group'][i]]
adamas['group_new'], adamas['group'] = adamas['group'], adamas['group_new']

In [17]:
adamas['metal'] = adamas['metal'].astype('int')

In [18]:
metal_dict = {
    3: 'Серебро 925',
    8: 'Золото 585',
    18: 'Не драгоценный',
    9: 'Золото 750'
}

In [19]:
for i in adamas['metal'].unique():
    if i not in metal_dict.keys():
        print(i)
        print(adamas[adamas['metal'] == i].head(1))
        print('='*100)

In [20]:
adamas['metal_new'] = np.nan
for i in adamas.index:
    adamas['metal_new'][i] = metal_dict[adamas['metal'][i]]
adamas['metal_new'], adamas['metal'] = adamas['metal'], adamas['metal_new']

In [21]:
color_metal_dict ={
    40: 'Красный',
    37: 'Белый',
    38: 'Желтый',
    0: 'Не определено'
}

In [22]:
adamas['color_metal'] = adamas['color_metal'].fillna(0).astype('int')

In [23]:
for i in adamas['color_metal'].unique():
    if i not in color_metal_dict.keys():
        print(i)

In [24]:
adamas['color_met'] = np.nan
for i in adamas.index:
    adamas['color_met'][i] = color_metal_dict[adamas['color_metal'][i]]
adamas['color_met'], adamas['color_metal'] = adamas['color_metal'], adamas['color_met']

In [25]:
adamas['proba'] = adamas['metal'].apply(lambda x: x.split(' ')[1])

In [26]:
adamas.group.unique()

array(['Кольца', 'Цепи', 'Серьги', 'Подвески', 'Браслеты', 'Колье',
       'Пирсинг', 'Броши'], dtype=object)

In [27]:
adamas['sizes'] = 'Нет информации'

In [28]:
adamas.loc[(adamas['insert'] == 397), 'insert'] = 'Без вставки'
adamas.loc[(adamas['insert'] == 106), 'insert'] = 'Фианит'
adamas.loc[(adamas['insert'] == '<без вставок>'), 'insert'] = 'Без вставки'

In [29]:
adamas['type_insert'] = np.where(
    adamas['insert'].str.lower().str.contains('бр|рубин|сапф|изумруд'), 'Драг', np.where(
    adamas['insert'].str.contains('Бр|БР'), 'Драг', np.where(
    adamas['insert'].str.lower().str.contains('аквамар|пер|ренит|лудраг|sky| гр |ончр|азурит|патит|арабин|ирит|топ|амет|ризокол|топлон|акрил|гранат|праз|марказ|жем|кварц|морганит|оникс|опал|перламутр|родолит|ситалл|султанит|топаз|турмалин|параиба|хризолит|цитрин|циркон|циркон|сваровски|александрит|шпинель|янтарь|керамика|агат|танзанит|хрусталь|муассанит|кристалл|коралл|малахит|смола|корунд|лабрадорит|лунный|бирюза|амазонит|авантюрин|вулканическая|ляпис|рубеллит|финифть|иолит|празиолит|дюмортьерит'), 'Полудраг', np.where(
    adamas['insert'].str.lower().str.contains('фиан|стекло|фианит'), 'Фианит', np.where(
    adamas['insert'].str.lower().str.contains('эмаль|без вставки'), 'Без вставки', 'другое'
)))))

In [30]:
adamas[adamas['type_insert'] == 'другое']['insert'].value_counts()

insert
белое золото                               5
1 Янт овал-кабошон 14.00*10.00             2
1 Мал площадка 10.00*7.00                  2
2 Мал площадка 10.00*7.00                  1
16 Турм овал 7.00*5.00,8 Турм круг 1.25    1
130 из (O) круг 1.25 3/3 0.764             1
130 Руб (F) круг 1.25 2/2 1.444            1
1 Swarovski Ф Круг 3.00 0.040 б/ц          1
Name: count, dtype: int64

In [31]:
adamas['type_insert'].value_counts()

type_insert
Драг           2848
Полудраг        730
Без вставки     493
Фианит          237
другое           14
Name: count, dtype: int64

In [32]:
for i in adamas[adamas['type_insert'] == 'другое'].index:
    if type(adamas['params'][i]['Вставка']) != int:
        adamas['insert'][i] = ', '.join([str(i) for i in adamas['params'][i]['Вставка']])
    else:
        adamas['insert'][i] = str(adamas['params'][i]['Вставка'])

In [33]:
adamas.loc[(adamas['insert'] == '397'), 'insert'] = 'Без вставки'
adamas.loc[(adamas['insert'] == '117'), 'insert'] = 'Янтарь'
adamas.loc[(adamas['insert'] == '78'), 'insert'] = 'Малахит'
adamas.loc[(adamas['insert'] == '79, 397'), 'insert'] = 'Полудрагоценный микс'
adamas.loc[(adamas['insert'] == '106, 397'), 'insert'] = 'Фианит'
adamas.loc[(adamas['insert'] == '105, 106'), 'insert'] = 'Турмалин, Фианит'

In [34]:
adamas['type_insert'] = np.where(
    adamas['insert'].str.lower().str.contains('бр|рубин|сапф|изумруд'), 'Драг', np.where(
    adamas['insert'].str.contains('Бр|БР'), 'Драг', np.where(
    adamas['insert'].str.lower().str.contains('аквамар|пер|ренит|лудраг|sky| гр |ончр|азурит|патит|арабин|ирит|топ|амет|ризокол|топлон|акрил|гранат|праз|марказ|жем|кварц|морганит|оникс|опал|перламутр|родолит|ситалл|султанит|топаз|турмалин|параиба|хризолит|цитрин|циркон|циркон|сваровски|александрит|шпинель|янтарь|керамика|агат|танзанит|хрусталь|муассанит|кристалл|коралл|малахит|смола|корунд|лабрадорит|лунный|бирюза|амазонит|авантюрин|вулканическая|ляпис|рубеллит|финифть|иолит|празиолит|дюмортьерит'), 'Полудраг', np.where(
    adamas['insert'].str.lower().str.contains('фиан|стекло|фианит'), 'Фианит', np.where(
    adamas['insert'].str.lower().str.contains('эмаль|без вставки'), 'Без вставки', 'другое'
)))))

In [35]:
adamas[adamas['type_insert'] == 'другое'].shape[0]

3

In [36]:
adamas['type_insert'].value_counts()

type_insert
Драг           2848
Полудраг        736
Без вставки     498
Фианит          237
другое            3
Name: count, dtype: int64

In [37]:
adamas_fin = adamas[[
    'date', 'shop', 'group', 'article', 'name', 'metal', 'color_metal', 'proba', 'weight', 'sizes', 'insert', 'type_insert', 
    'params', 'price_old', 'price', 'url'
]]

In [38]:
adamas_fin.columns = [
    'Дата парсинга', 'Наименование конкурента', 'Группа товаров', 'Артикул', 'Название изделия', 'Материал', 'Цвет металла', 
    'Проба', 'Вес', 'Размерный ряд', 'Вставка', 'Тип вставки', 'Прочая информация', 'Цена до скидок', 'Цена со скидками', 
    'Ссылка'

]

In [39]:
adamas_fin.head()

,Дата парсинга,Наименование конкурента,Группа товаров,Артикул,Название изделия,Материал,Цвет металла,Проба,Вес,Размерный ряд,Вставка,Тип вставки,Прочая информация,Цена до скидок,Цена со скидками,Ссылка
0,14.03.2024,Adamas,Кольца,1212150-А501Ф-01,Кольцо,Золото 585,Красный,585,3.30,Нет информации,Без вставки,Без вставки,"{'Категория': 304, 'Тип кольца': 319, 'Цвет металла': 40, 'Металл': 8, 'Вставка': 397, 'Тематика': 2592, 'Вес': '3.3'}",97290,58374,https://adamas.ru/catalog/rings/1212150-A501F-01/
1,14.03.2024,Adamas,Цепи,ЦГРП250СА6-А51,"Цепь, плетение Гарибальди",Золото 585,Красный,585,7.91,Нет информации,Без вставки,Без вставки,"{'Категория': 302, 'Вид плетения': 256, 'Плетение': 145, 'Цвет металла': 40, 'Металл': 8, 'Вставка': 397, 'Обработка...",60990,60990,https://adamas.ru/catalog/chains/TSGRP250SA6-A51/
2,14.03.2024,Adamas,Кольца,12039153/01-А50-01,Кольцо,Золото 585,Красный,585,1.81,Нет информации,Без вставки,Без вставки,"{'Категория': 304, 'Тип кольца': 319, 'Цвет металла': 40, 'Металл': 8, 'Вставка': 397, 'Пол': 291, 'Тематика': 2592,...",56690,34014,https://adamas.ru/catalog/rings/12039153-01-A50-01/
3,14.03.2024,Adamas,Кольца,1254553/01-А50Д-41,Обручальное кольцо из красного золота 585 пробы с бриллиантом,Золото 585,Красный,585,2.38,Нет информации,3 Бриллиант Круг 57 0.009 200-400 4/6А;,Драг,"{'Категория': 304, 'Тип кольца': 319, 'Цвет металла': 40, 'Металл': 8, 'Вставка': 59, 'Обработка': 296, 'Вид': [1260...",162490,64996,https://adamas.ru/catalog/rings/1254553-01-A50D-41/
4,14.03.2024,Adamas,Серьги,2457923/01-А507Д-41,Серьги из красного золота 585 пробы с бриллиантом,Золото 585,Красный,585,3.71,Нет информации,"40 бриллианты 0,11802ct 2/3А кр-17 ; 2 бриллианты 0,05782ct 4/6А кр-57",Драг,"{'Категория': 305, 'Тип серег': 321, 'Цвет металла': 40, 'Металл': 8, 'Вставка': 59, 'Обработка': 294, 'Пол': 291, '...",64490,64490,https://adamas.ru/catalog/earrings/2457923-01-A507D-41/


In [40]:
adamas_fin.to_csv(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\parsed_data\adamas_all_{day_now}_{month_now}_{year_now}.csv', index=False)

In [41]:
adamas_fin.to_csv(rf'C:\Users\Trenkin.Sergey\Desktop\parsers\workspace\adamas_all_{day_now}_{month_now}_{year_now}.csv', index=False)